# Plot generator
Il programma apre un file .txt di misure di sweep di tensione, ricostruisce le liste di misure originali, le plotta e ne salva le immagini. Ciascun plot può essere sovrapposto o affiancato. 


In [ ]:
# Required imports
import sys
username = 'admin' # 'admin' for BO1 lab
sys.path.append('c:/users/'+ username +'/miniconda3/lib/site-packages')
import matplotlib.pyplot as plt
import qontrol
import time
import numpy as np
import datetime
import os

# Set the right path and file
lines = [] # list of lines of the .txt file
file_name = "Multiple_channels_control_20_01_2021_16_57_29.txt"

# Variable declarations
F_overlapping_plots = 1 # 0 for non overlapping plots, not 0 otherwise
voltage_start = [] # list of voltage starting values for each channel/try
voltage_stop = []  # list of voltage stopping values for each channel/try
voltage_step = []  # list of voltage steps for each channel/try

In [ ]:
def from_linenumb_to_active_channel(line_numb, channel_steps):
    """
    map the line number to the active channel number. 
    N.B: the active channel is the one under sweeping
    """
    total_steps = 0
    for counter in range(len(channel_steps)):
        if (line_numb >= total_steps) and (line_numb < total_steps+channel_steps[counter]):
            return counter # the counter is exactly the active channel 
        total_steps += channel_steps[counter]        

In [ ]:
#change to the target directory  
os.chdir('./Log') 

# sequentially open each file and perform information extraction and data visualization
for j in os.listdir(): 

    # Variable re-initialization
    lines = []
    voltage_start = [] 
    voltage_stop = []  
    voltage_step = []  
    
    file1 = open(j,"r") 
    filename = file1.name.strip(".txt")

    # extract the voltage_start, voltage_stop, voltage_step and triangular from the file
    for i in range(9):
        line = file1.readline()
        if i == 5:
            # split into individual values and stripping the % and the \n
            voltage_start = [float(s.strip('%')) for s in line.rstrip('\n').split('\t')] 
        elif i == 6:
            voltage_stop = [float(s.strip('%')) for s in line.rstrip('\n').split('\t')]
        elif i == 7:
            voltage_step = [float(s.strip('%')) for s in line.rstrip('\n').split('\t')]
        elif i == 8:
            F_triangular = int(line.rstrip('\n').strip('%')) # FLAG : 0 for ramp sweep, 1 for triangular sweep
        else:
            pass        

    # save each line as a list in the 'lines' list 
    for line in file1:
        if line[0] != '%':
            lines.append(line.rstrip('\n').split('\t'))

    # compute the number of channels/tries
    channels = int(len(lines[0])/2-1) # /2 because they are grouped in couples (current-voltages), 
                                      # -1 because first two columns are for PD
    
    # compute the total steps for each channel/try        
    channel_steps = [int((F_triangular+1)*(voltage_stop[j]-voltage_start[j])/voltage_step[j]) for j in range(channels)]

    # re-build the original measured_voltage, measured_current, PD_voltage and PD_current
    # measured_voltage[channels][channel_under_sweep][measurement]
    measured_voltage = [[[] for i in range(channels)] for j in range(channels)] 
    measured_current = [[[] for i in range(channels)] for j in range(channels)]
    PD_voltage = [[] for i in range(channels)]
    PD_current = [[] for i in range(channels)]

    # populate each list
    for i in range(len(lines)):
        for channel in range(0,len(lines[0]),2):
            if channel == 0: # first two columns of the file are for the PD voltage and current
                PD_voltage[from_linenumb_to_active_channel(i, channel_steps)].append(float(lines[i][channel]))
                PD_current[from_linenumb_to_active_channel(i, channel_steps)].append(float(lines[i][channel+1]))
            else: # remaining columns are for the actual channels
                measured_voltage[int((channel/2)-1)][from_linenumb_to_active_channel(i, channel_steps)].append(float(lines[i][channel]))
                measured_current[int((channel/2)-1)][from_linenumb_to_active_channel(i, channel_steps)].append(float(lines[i][channel+1]))          

    # Close the log file
    file1.close()

    # Plot results
    fig, axs = plt.subplots(2,1)
    fig.suptitle(filename)
    
    axs[0].set_title('Channel Voltage vs  Channel Current ')
    axs[0].set_xlabel('Voltage [V]')
    axs[0].set_ylabel('Current [mA]')
    axs[1].set_title('Channel Voltage vs  -PD Voltage ')
    axs[1].set_xlabel('Voltage [V]')
    axs[1].set_ylabel('Voltage [mV]')

    for i in range(channels):
        #print("\nPLOTS FOR DRIVING CHANNEL {:} \n" .format(channels[i]))
        if F_overlapping_plots == 0:
            axs[0].plot(range(i*len(measured_current[i][i]),(i+1)*len(measured_current[i][i])), measured_current[i][i])
            axs[1].plot(range(i*len(measured_current[i][i]),(i+1)*len(measured_current[i][i])), PD_voltage[i])
        else:
            axs[0].plot(measured_voltage[i][i], measured_current[i][i])
            axs[1].plot(range(len(measured_current[i][i])), PD_voltage[i])
        #plt.xticks(np.arange(min(measured_current[i][i]), max(measured_current[i][i])))
        #plt.xticks(np.arange(min(PD_current[i]), max(PD_current[i])+1))

    plt.tight_layout()
    plt.savefig('../Figures/' + filename + '.png')